# ApplyVortex: Qwen 2.5-7B Fine-Tuning (Colab Edition)

This notebook is optimized for **Google Colab T4 GPUs**. It includes Google Drive integration for persistent storage of your fine-tuned model.

In [ ]:
%%capture
# 1. COLAB INSTALLATION
# Using the [colab-new] tag for optimized T4 kernels
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

### **IMPORTANT**: After the cell above finishes, click the **"Restart Session"** button at the bottom of the output. Then continue below.

In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import drive

# MOUNT GOOGLE DRIVE (Recommended for saving your model)
drive.mount('/content/drive')

MAX_SEQ_LENGTH = 8192 # Supporting long CVs
dtype = None # Auto detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank 16 fits comfortably in Colab T4 VRAM
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("✅ Model and Adapters loaded successfully!")

In [ ]:
from google.colab import files
from datasets import load_dataset
import os

# UPLOAD DATASET
if not os.path.exists("applyvortex_qwen_train.jsonl"):
    print("Select your 'applyvortex_qwen_train.jsonl' file to upload:")
    uploaded = files.upload()

dataset = load_dataset("json", data_files="applyvortex_qwen_train.jsonl", split="train")
print(f"🚀 Loaded {len(dataset)} training examples.")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60, # Start small to verify; increase to 300+ for real training
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
# SAVE TO GOOGLE DRIVE
SAVE_PATH = "/content/drive/MyDrive/ApplyVortex-Qwen-Adapter"
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"💾 Fine-tuned adapter saved to: {SAVE_PATH}")